<div class="alert alert-block alert-warning">The end result of this exercise should be a file named acquire.py.</div>

In [1]:
import requests
import pandas as pd

1. Using the code from the lesson as a guide, create a dataframe named items that has all of the data for items.

In [3]:
# Load the data in JSON

response = requests.get('https://python.zach.lol/api/v1/items')
items= response.json()
items.keys()

dict_keys(['payload', 'status'])

In [5]:
# Print the keys of 'payload'
items['payload'].keys()

dict_keys(['items', 'max_page', 'next_page', 'page', 'previous_page'])

In [7]:
# Print the value of 'status' since 'status' doesn't have nested keys
items['status']

'ok'

In [9]:
# How many items in the data
len(items['payload']['items'])

20

In [10]:
# How many pages in the data?
items['payload']['max_page']

3

In [12]:
# What is the value of 'next_page'
items['payload']['next_page']

'/api/v1/items?page=2'

In [13]:
# What is the value of 'page'?
items['payload']['page']

1

In [28]:
# Create a for loop to combine all the pages

# Create a base_url
base_url = 'https://python.zach.lol'

# Load the first page of the data
response = requests.get(base_url + '/api/v1/items')
items = response.json()

# Conver the data to df
df_items = pd.DataFrame(items['payload']['items'])

# For loop the rest of the pages

pages = 2
for page in range(0, pages):
    response = requests.get(base_url + items['payload']['next_page'])
    items = response.json()
    df_items = pd.concat([df_items, pd.DataFrame(items['payload']['items'])]).reset_index()

df_items.shape

(50, 8)

In [10]:
# No reset_index()

df.head()

,item_brand,item_id,item_name,item_price,item_upc12,item_upc14
0,Riceland,1,Riceland American Jazmine Rice,0.84,35200264013,35200264013
1,Caress,2,Caress Velvet Bliss Ultra Silkening Beauty Bar...,6.44,11111065925,11111065925
2,Earths Best,3,Earths Best Organic Fruit Yogurt Smoothie Mixe...,2.43,23923330139,23923330139
3,Boars Head,4,Boars Head Sliced White American Cheese - 120 Ct,3.14,208528800007,208528800007
4,Back To Nature,5,Back To Nature Gluten Free White Cheddar Rice ...,2.61,759283100036,759283100036


In [12]:
# Reset_index()

df.head()

,level_0,index,item_brand,item_id,item_name,item_price,item_upc12,item_upc14
0,0,0.0,Riceland,1,Riceland American Jazmine Rice,0.84,35200264013,35200264013
1,1,1.0,Caress,2,Caress Velvet Bliss Ultra Silkening Beauty Bar...,6.44,11111065925,11111065925
2,2,2.0,Earths Best,3,Earths Best Organic Fruit Yogurt Smoothie Mixe...,2.43,23923330139,23923330139
3,3,3.0,Boars Head,4,Boars Head Sliced White American Cheese - 120 Ct,3.14,208528800007,208528800007
4,4,4.0,Back To Nature,5,Back To Nature Gluten Free White Cheddar Rice ...,2.61,759283100036,759283100036


2. Do the same thing, but for stores. 

In [18]:
# What is the key structure for stores?

response = requests.get('https://python.zach.lol/api/v1/stores')

stores = response.json()
stores.keys()

dict_keys(['payload', 'status'])

In [19]:
# Any nested key in 'payload'?

stores['payload'].keys()

dict_keys(['max_page', 'next_page', 'page', 'previous_page', 'stores'])

In [23]:
# What is the max page in stores/payload/payload?
# max page is 1

stores['payload']['max_page']

1

In [25]:
# Conver the data to df

df_stores = pd.DataFrame(stores['payload']['stores'])
df_stores.shape

(10, 5)

3. Extract the data for sales. There are a lot of pages of data here, so your code will need to be a little more complex. Your code should continue fetching data from the next page until all of the data is extracted.